# COVID-19 Prediction with ARIMA

## 1. Data Pre-processing

We first define some helper classes and functions to aid in our data pre-processing. Using the `numpy` and `pandas` library, we provide ourselves with some useful and needed functionalities such as parsing the input data into a dataframe, dropping unneeded features and accessing certain needed features from the input data.

The following code blocks in 1.1 should be run first.

### 1.1 Constants, Helper Functions, and Helper Classes

In [ ]:
data_round1_dir = 'data/round1'
data_round2_dir = 'data/round2'
submissions_dir = 'data/submission'

In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
# https://gist.github.com/rogerallen/1583593
# ^ Use this in case we need state code translation

from sklearn.preprocessing import StandardScaler

import math
import matplotlib.pyplot as plt
class utils:
    def __init__(self):
        pass
    
    def split(self, dataframe, test=0.2):
        size = dataframe.shape
        train_size = size[0] - math.floor(size[0] * test)
        return dataframe[:train_size], dataframe[train_size:]
    
class CoreData(object):
    def __init__(self, data_path = None, debug=False):
        if not data_path:
            raise Exception("Input file path!")
        self.data_path = data_path
        self.df = None
        self.states = []
        self.state_df = {}
        self.debug = debug
        
    def load(self, normalize=True):
        '''
        DataProcessor.load()
        Pre-load data state-by-state to a dictionary.
        '''
        # Change this line to modify dropped data series.
        dropped_col = ['ID', 'Province_State', 'Date', 'Incident_Rate', 'Recovered', 'People_Tested', 'People_Hospitalized', 'Mortality_Rate', 'Testing_Rate', 'Hospitalization_Rate' ]
        
        self.df = pd.read_csv(train_data_path)
        self.states = list(np.unique(self.df['Province_State']))
        self.state_df = dict.fromkeys(self.states, None)
        self.mean = dict.fromkeys(self.states, None)
        self.std = dict.fromkeys(self.states, None)
        
        for s in self.states:
            df_filter = self.df['Province_State'] == s
            tmp_state_df = self.df[df_filter]
            self.state_df[s] = tmp_state_df.drop(dropped_col, 1)
            
            # Normalize
            if normalize:
                self.mean[s] = self.state_df[s].mean()
                self.std[s] = self.state_df[s].std()
                self.state_df[s] = (self.state_df[s] - self.mean[s])/self.std[s]
    
    def access(self, state=None):
        if not state:
            raise Exception('Enter state name! i.e. self.access("California")')
        elif state not in self.states:
            raise Exception('Check your spelling of the state.')
        return self.state_df[state]
    
    def access_split(self, state=None, test_portion=0.2):
        df = self.access(state)
        return utils().split(dataframe=df, test=test_portion)


## 2. Round 1 Forecasting

In this section, we build and train an ARIMA model to predict COVID-19 confirmed cases and deaths for each state between Sept 1 and Sept 26, using data from April 12 to August 31. Each state will have its own model and its own set of hyperparameters, as the trends can be wildly different for different states. Data for round 1 is located in the `data/round1/` directory.

### 2.1 Load Round 1 Data 

In [3]:
import math
from pandas.plotting import lag_plot
from datetime import datetime
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
from pandas.plotting import autocorrelation_plot
import warnings
warnings.filterwarnings('ignore')

In [4]:
train_data_path = os.path.join(data_round1_dir, "train.csv")
test_data_path  = os.path.join(data_round1_dir, "test.csv")
graph_data_path = os.path.join(data_round1_dir, "graph.csv")
dp = CoreData(train_data_path)
dp.load(False)

### 2.2 Forecasting Confirmed Cases

In [5]:
N_test = 26

# states in outliers1 are predited to have lower value than the ground truth from our cross-validation reults.
# states in outliers2 are predited to have higher value than the ground truth from our cross-validation reults.
outliers1 = ['West Virginia', 'North Dakota','Wisconsin','Utah', 'Wyoming', 'Montana']
outliers2 = ['Alabama', 'Vermont']

result_confirmed = {}
states = dp.states
for state in states:
    print(f"Processing: {state}...")
    training_data, test_data = dp.access_split(state, 0)
    data = dp.access(state)
    training = training_data['Confirmed'].values
    testing = test_data['Confirmed'].values

    # Use the differences between two days to train our model
    temp_data = []
    for i in range(1, len(training)):
        temp_data.append(training[i] - training[i-1])
    training = temp_data

    #Starting training
    history = [x for x in training]
    model_predictions = []
    for time in range(N_test):
        # Choose parameters (p,d,q) for ARIMA depended on the population of the state
        # and use larger lag 8 for long-period prediction
        try:
            if sum(training) / len(training) > 1500:
                model = ARIMA(history, order=(8,2,1))
            elif time < 14:
                model = ARIMA(history, order=(1,1,1))
            else:
                model = ARIMA(history, order=(8,2,1))
            model_fit = model.fit(disp=0)
        except:
            # In case the model cannot converge with the larger lag parameters.
            model = ARIMA(history, order=(1,1,0))
            model_fit = model.fit(disp=0)

        output = model_fit.forecast()
        yhat = output[0]
        # Force the result to at least increase by a small value to avoid predictions of all-zeros.
        if yhat[0] < 0.5:
            yhat[0]=0.5

        # Hawaii is a speacial case that cause very larger error, so it has differnet treatment.
        if state == 'Hawaii':
            if time < 14:
                yhat[0] = 150
            else:
                yhat[0] = 100
        
        model_predictions.append(yhat)
        true_test_value = yhat
        # Feedback the result to history for the next prediction
        history.append(true_test_value)
    
    # Process the results
    M = []
    for m in model_predictions:
        M.append(m.tolist()[0])
    
    # Restore the values from differences
    training.insert(0, training_data['Confirmed'].values[0])
    for i in range(1, len(training)):
        training[i] = training[i-1] + training[i]
    M[0] = training[-1] + M[0]
    for i in range(1, len(M)):
        M[i] = M[i-1] + M[i]
        if i >=16:
            # Process the outliers
            if state in outliers1:
                M[i] = M[i] + M[0] / 50
            elif state in outliers2:
                M[i] = M[i] - M[0] / 100
    print(f"Done: {state}...{M}")
    
    result_confirmed[state] = M


7879808255, 106054.81180025324, 107067.83725921025, 108084.55539645533, 109120.11115601542, 110162.69706446465, 111213.21417054739, 112269.78264523136, 113330.44749319852, 114396.66875928166, 115471.31080446752, 116555.83618019786, 117648.6054775074, 118749.94872895023, 119857.94431760091, 120972.29399256792]
Processing: Iowa...
Done: Iowa...[65750.54199960543, 66392.83050751635, 67050.08147682124, 67715.47431959322, 68386.06757102373, 69060.5926909687, 69738.50265879082, 70419.56151017631, 71103.66803556815, 71790.7783527692, 72480.87382282023, 73173.94591135545, 73869.99131728862, 74569.00821186995, 75266.63535851707, 75965.07914532922, 76665.15789932669, 77366.95551105398, 78070.91182625205, 78777.0253924424, 79484.31238212512, 80192.27177495798, 80901.26696641433, 81611.54600998765, 82323.29084951432, 83036.6240227166]
Processing: Kansas...
Done: Kansas...[43035.1008292906, 43673.35350426799, 44217.7176930893, 44813.28773390899, 45388.9450556241, 45979.54994324108, 46568.0135411585

### 2.3 Forecasting Deaths

In [6]:
N_test = 26

# states in outliers1 are predited to have lower value than the ground truth from our cross-validation reults.
# states in outliers2 are predited to have higher value than the ground truth from our cross-validation reults.
outliers1 = ['Missouri', 'Kansas', 'North Dakota', 'Montana', 'North Dakota', 'Ohio', 'Kentucky', 'Arizona', 'Florida', 'South Dakota', 'Utah']
outliers2 = ['Mississippi', 'Alabama', 'Idaho', 'Georgia', 'New York', 'Indiana', 'Oregon', 'Nebraska','Michigan','South Carolina']
result_death = {}
states = dp.states
for state in states:
    print(f"Processing: {state}...")
    training_data, test_data = dp.access_split(state, 0)
    data = dp.access(state)
    training = training_data['Deaths'].values
    testing = test_data['Deaths'].values

    # Use the differences between two days to train our model
    temp_data = []
    for i in range(1, len(training)):
        temp_data.append(training[i] - training[i-1])
    training = temp_data

    #Starting training
    history = [x for x in training]
    model_predictions = []
    for time in range(N_test):
        # Choose parameters (p,d,q) for ARIMA depended on the population of the state
        # and use larger lag 6 for long-period prediction
        try:
            if sum(training) / len(training) > 30:
                model = ARIMA(history, order=(6,2,1))
            elif time < 14:
                model = ARIMA(history, order=(1,1,1))
            else:
                model = ARIMA(history, order=(6,2,1))
            model_fit = model.fit(disp=0)
        except:
            # In case the model cannot converge with the larger lag parameters.
            model = ARIMA(history, order=(1,1,0))
            model_fit = model.fit(disp=0)
        output = model_fit.forecast()
        yhat = output[0]

        # Force the result to at least increase by a small value to avoid predictions of all-zeros.
        if yhat[0] < 0.5:
            yhat[0]=0.5

        # Hawaii and Vermont are speacial cases that cause very larger error, so they have differnet treatment.
        if state == 'Hawaii':
            if time < 18:
                if yhat[0] > 2:
                    yhat[0] = 2
            else:
                yhat[0] = 3
        if state == 'Vermont':
            yhat[0] = 0


        model_predictions.append(yhat)
        true_test_value = yhat

        # Feedback the result to history for the next prediction
        history.append(true_test_value)

    # Process the results
    M = []
    for m in model_predictions:
        M.append(m.tolist()[0])

    # Restore the values from differences
    training.insert(0, training_data['Deaths'].values[0])
    for i in range(1, len(training)):
        training[i] = training[i-1] + training[i]
    M[0] = training[-1] + M[0]
    for i in range(1, len(M)):
        # Process the outliers
        if state in outliers1:
            M[i] += M[i-1] + M[i] * 1.05
        elif state in outliers2:
            M[i] += M[i-1] + M[i] * 0.05
        else:
            M[i] = M[i-1] + M[i]
    print(f"Done: {state}...{M}")
    
    result_death[state] = M


5, 8446.18209288335, 8450.265473211075, 8453.790903931444, 8455.497483970197, 8455.997483970197, 8456.497483970197]
Processing: Indiana...
20.94326241134752
Done: Indiana...[3298.495867978936, 3300.2389888553685, 3301.5941957539094, 3302.651548788358, 3303.431829131075, 3303.9398319258985, 3304.4448319258986, 3304.9498319258987, 3305.454831925899, 3305.959831925899, 3306.464831925899, 3306.969831925899, 3307.4748319258993, 3307.9798319258994, 3308.4848319258995, 3308.9898319258996, 3309.4948319258997, 3309.9998319259, 3310.5048319259, 3311.0098319259, 3311.5148319259, 3312.0198319259002, 3312.5248319259003, 3313.0298319259005, 3313.5348319259006, 3314.0398319259007]
Processing: Iowa...
7.652482269503546
Done: Iowa...[1129.738500094612, 1138.3234525750793, 1147.4884262095186, 1156.4612273939426, 1165.5858345170218, 1174.7848796323765, 1184.0174816847589, 1193.3018542699397, 1202.629882088944, 1211.9483341029863, 1221.2949718694608, 1230.668749963929, 1240.0696946759358, 1249.49780325143

### 2.4 Generate Submission Output File

In [7]:
forecastID = [x for x in range(N_test*50)]
deaths = []
confirmed = []

for i in range(N_test):
    for s in states:
        print(f"Day {i}/{N_test}, {s}")
        confirmed.append(result_confirmed[s][i])
        deaths.append(result_death[s][i])



Day 6/26, Indiana
Day 6/26, Iowa
Day 6/26, Kansas
Day 6/26, Kentucky
Day 6/26, Louisiana
Day 6/26, Maine
Day 6/26, Maryland
Day 6/26, Massachusetts
Day 6/26, Michigan
Day 6/26, Minnesota
Day 6/26, Mississippi
Day 6/26, Missouri
Day 6/26, Montana
Day 6/26, Nebraska
Day 6/26, Nevada
Day 6/26, New Hampshire
Day 6/26, New Jersey
Day 6/26, New Mexico
Day 6/26, New York
Day 6/26, North Carolina
Day 6/26, North Dakota
Day 6/26, Ohio
Day 6/26, Oklahoma
Day 6/26, Oregon
Day 6/26, Pennsylvania
Day 6/26, Rhode Island
Day 6/26, South Carolina
Day 6/26, South Dakota
Day 6/26, Tennessee
Day 6/26, Texas
Day 6/26, Utah
Day 6/26, Vermont
Day 6/26, Virginia
Day 6/26, Washington
Day 6/26, West Virginia
Day 6/26, Wisconsin
Day 6/26, Wyoming
Day 7/26, Alabama
Day 7/26, Alaska
Day 7/26, Arizona
Day 7/26, Arkansas
Day 7/26, California
Day 7/26, Colorado
Day 7/26, Connecticut
Day 7/26, Delaware
Day 7/26, Florida
Day 7/26, Georgia
Day 7/26, Hawaii
Day 7/26, Idaho
Day 7/26, Illinois
Day 7/26, Indiana
Day 7/26,

In [8]:
round1_path = os.path.join(submissions_dir, 'Team4_Round1.csv')
final = pd.DataFrame(list(zip(forecastID, confirmed, deaths)), 
               columns =['ForecastID', 'Confirmed', 'Deaths']) 
final.to_csv(round1_path, index=False)

## 3. Round 2 Forecasting

In this section, we build and train an ARIMA model to predict COVID-19 confirmed cases and deaths for each state between Dec 7 and Dec 13, using data from April 12 to December 5. The data from April 12 to November 22 is given, and the data from November 23 to December 5 is obtained by crawling Johns Hopkins University's dataset for confirmed COVID-19 cases and deaths. Each state will have its own model and its own set of hyperparameters, with further tweaks from round 1, as the trends can be wildly different for different states. Data for round 1 is located in the `data/round2/` directory.

### 3.1 Load Round 2 Data

In [9]:
import math
from pandas.plotting import lag_plot
from datetime import datetime
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
from pandas.plotting import autocorrelation_plot
import warnings
warnings.filterwarnings('ignore')

In [10]:
train_data_path = os.path.join(data_round2_dir, "train_round2-1205.csv")
test_data_path  = os.path.join(data_round2_dir, "test_round2.csv")
graph_data_path = os.path.join(data_round2_dir, "graph_round2.csv")
dp = CoreData(train_data_path)
dp.load(False)

### 3.2 Forecasting Confirmed Cases

In [11]:
N_test = 7

result_confirmed = {}
states = dp.states
for state in states:
    print(f"Processing: {state}...")
    training_data, test_data = dp.access_split(state, 0)
    data = dp.access(state)
    training = training_data['Confirmed'].values
    testing = test_data['Confirmed'].values

    # Use the differences between two days to train our model
    temp_data = []
    for i in range(1, len(training)):
        temp_data.append(training[i] - training[i-1])
    training = temp_data
    
    #Starting training
    history = [x for x in training]
    model_predictions = []
    for time in range(N_test):
        # Choose parameters (p,d,q) for ARIMA depended on the population of the state
        # and use larger lag 8 for long-period prediction
        try:
            if sum(training) / len(training) > 1500:
                model = ARIMA(history, order=(8,2,1))
            elif time < 14:
                model = ARIMA(history, order=(1,1,1))
            else:
                model = ARIMA(history, order=(8,2,1))
            model_fit = model.fit(disp=0)
        except:
            # In case the model cannot converge with the larger lag parameters.
            model = ARIMA(history, order=(1,1,0))
            model_fit = model.fit(disp=0)
        
        output = model_fit.forecast()
        yhat = output[0]
        # Force the result to at least increase by a small value to avoid predictions of all-zeros.
        if yhat[0] < 0.5:
            yhat[0]=0.5
        model_predictions.append(yhat)
        true_test_value = yhat

        # Feedback the result to history for the next prediction
        history.append(true_test_value)

    # Process the results
    M = []
    for m in model_predictions:
        M.append(m.tolist()[0])
    
    # Restore the values from differences
    training.insert(0, training_data['Confirmed'].values[0])
    for i in range(1, len(training)):
        training[i] = training[i-1] + training[i]
    M[0] = training[-1] + M[0]
    for i in range(1, len(M)):
        M[i] = M[i-1] + M[i]
    print(f"Done: {state}...{M}")
    
    result_confirmed[state] = M[-7:]

Processing: Alabama...
Done: Alabama...[270765.66579695867, 273913.8796063688, 277067.2217349944, 280231.7917991511, 283408.7043617718, 286598.16443212336, 289800.2069186907]
Processing: Alaska...
Done: Alaska...[37024.6648364356, 37772.67679436186, 38523.44865938673, 39277.360530716796, 40034.43064904476, 40794.66030019774, 41558.05013065874]
Processing: Arizona...
Done: Arizona...[364347.9467338932, 370109.31413161044, 375832.9999577312, 381593.078751198, 387373.9193127212, 393178.8235872269, 399007.0592470166]
Processing: Arkansas...
Done: Arkansas...[171745.98596079682, 173917.44149011455, 176012.5925977049, 178080.42245343694, 180141.57450630137, 182204.75072963882, 184273.61891767624]
Processing: California...
Done: California...[1363236.4814962118, 1389739.0190118572, 1417096.863298124, 1447309.701215487, 1479037.362354616, 1514064.1117877793, 1550171.970116632]
Processing: Colorado...
Done: Colorado...[262386.4185934277, 267409.76666780683, 272440.7068307425, 277487.3069017269,

### 3.3 Forecasting Deaths

In [12]:
N_test = 7

result_death = {}
states = dp.states
for state in states:
    print(f"Processing: {state}...")
    training_data, test_data = dp.access_split(state, 0)
    data = dp.access(state)
    training = training_data['Deaths'].values
    testing = test_data['Deaths'].values
    temp_data = []
    for i in range(1, len(training)):
        temp_data.append(training[i] - training[i-1])
    training = temp_data
    history = [x for x in training]
    model_predictions = []
    print(sum(training) / len(training))
    for time in range(N_test):
        # Choose parameters (p,d,q) for ARIMA depended on the population of the state
        # and use larger lag 6 for long-period prediction
        try:
            if sum(training) / len(training) > 30:
                model = ARIMA(history, order=(6,2,1))
            elif time < 14:
                model = ARIMA(history, order=(1,1,1))
            else:
                model = ARIMA(history, order=(6,2,1))
            model_fit = model.fit(disp=0)
        except:
            # In case the model cannot converge with the larger lag parameters.
            model = ARIMA(history, order=(1,1,0))
            model_fit = model.fit(disp=0)
        output = model_fit.forecast()
        yhat = output[0]

        # Force the result to at least increase by a small value to avoid predictions of all-zeros.
        if yhat[0] < 0.5:
            yhat[0]=0.5

        model_predictions.append(yhat)
        true_test_value = yhat
        # Feedback the result to history for the next prediction
        history.append(true_test_value)

    # Process the results
    M = []
    for m in model_predictions:
        M.append(m.tolist()[0])

    # Restore the values from differences
    training.insert(0, training_data['Deaths'].values[0])   
    for i in range(1, len(training)):
        training[i] = training[i-1] + training[i]
    M[0] = training[-1] + M[0]
    for i in range(1, len(M)):
        M[i] = M[i-1] + M[i]
    print(f"Done: {state}...{M}")
    
    result_death[state] = M[-7:]

Processing: Alabama...
15.966244725738397
Done: Alabama...[3902.9744143363687, 3944.594792020825, 3982.3533408205535, 4014.942213856504, 4046.3898530175093, 4078.6587379549924, 4111.612816810477]
Processing: Alaska...
0.569620253164557
Done: Alaska...[144.93666132979374, 146.47453773935626, 148.01847063974685, 149.55156340378358, 151.09039749571417, 152.63600171789943, 154.1892411769681]
Processing: Arizona...
28.734177215189874
Done: Arizona...[6962.761056156777, 7004.701774690392, 7047.264307163191, 7089.281243047406, 7131.331918239243, 7173.627274415188, 7216.073480043679]
Processing: Arkansas...
10.940928270042194
Done: Arkansas...[2642.7841905843907, 2665.4465628633848, 2688.5747754749955, 2711.816647974534, 2735.14756515728, 2758.5786348424444, 2782.1108320540266]
Processing: California...
81.13924050632912
Done: California...[19909.82647215438, 19982.32593034745, 20094.846507309878, 20238.19453158905, 20393.463027350732, 20575.532070225127, 20715.855282983015]
Processing: Colora

### 3.4 Generate Submission Output File

In [13]:
forecastID = [x for x in range(N_test*50)]
deaths = []
confirmed = []
for i in range(7):
    for s in states:
        print(f"Day {i}/{7}, {s}")
        confirmed.append(result_confirmed[s][i])
        deaths.append(result_death[s][i])


Day 0/7, Alabama
Day 0/7, Alaska
Day 0/7, Arizona
Day 0/7, Arkansas
Day 0/7, California
Day 0/7, Colorado
Day 0/7, Connecticut
Day 0/7, Delaware
Day 0/7, Florida
Day 0/7, Georgia
Day 0/7, Hawaii
Day 0/7, Idaho
Day 0/7, Illinois
Day 0/7, Indiana
Day 0/7, Iowa
Day 0/7, Kansas
Day 0/7, Kentucky
Day 0/7, Louisiana
Day 0/7, Maine
Day 0/7, Maryland
Day 0/7, Massachusetts
Day 0/7, Michigan
Day 0/7, Minnesota
Day 0/7, Mississippi
Day 0/7, Missouri
Day 0/7, Montana
Day 0/7, Nebraska
Day 0/7, Nevada
Day 0/7, New Hampshire
Day 0/7, New Jersey
Day 0/7, New Mexico
Day 0/7, New York
Day 0/7, North Carolina
Day 0/7, North Dakota
Day 0/7, Ohio
Day 0/7, Oklahoma
Day 0/7, Oregon
Day 0/7, Pennsylvania
Day 0/7, Rhode Island
Day 0/7, South Carolina
Day 0/7, South Dakota
Day 0/7, Tennessee
Day 0/7, Texas
Day 0/7, Utah
Day 0/7, Vermont
Day 0/7, Virginia
Day 0/7, Washington
Day 0/7, West Virginia
Day 0/7, Wisconsin
Day 0/7, Wyoming
Day 1/7, Alabama
Day 1/7, Alaska
Day 1/7, Arizona
Day 1/7, Arkansas
Day 1/7, C

In [14]:
round2_path = os.path.join(submissions_dir, 'Team4_Round2.csv')
final = pd.DataFrame(list(zip(forecastID, confirmed, deaths)), 
               columns =['ForecastID', 'Confirmed', 'Deaths']) 
final.to_csv(round2_path, index=False)

# End of Notebook